In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/

# classification

In [ ]:
import os
import glob
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.models import ResNet34_Weights
from sklearn.metrics import confusion_matrix

In [ ]:
# trainset
train_path = './SVD/mel_spectrograms/train'
trainset = ImageFolder(root=train_path, transform=transforms.Compose([transforms.ToTensor(),
                                                                     transforms.Resize((224, 224))]))
#print(trainset)
print(f'\nclass : index\n{trainset.class_to_idx}')

In [ ]:
# testset
test_path = './SVD/mel_spectrograms/test'
testset = ImageFolder(root=test_path, transform=transforms.Compose([transforms.ToTensor(),
                                                                    transforms.Resize((224, 224))]))
#print(testset)
print(f'\nclass : index\n{testset.class_to_idx}')

In [ ]:
# dataloader
train_dataloader = DataLoader(trainset, batch_size=4, shuffle=True, num_workers=4)
test_dataloader = DataLoader(testset, batch_size=4, shuffle=False, num_workers=4)

In [ ]:
# GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

In [ ]:
!pip install efficientnet-pytorch

In [ ]:
# A model includes 20 effnet models
# When test five times, the seed is 1000*n, 10000*n, not 100*n, so that the seed does not overlapfrom efficientnet_pytorch
import EfficientNet
for num in range(19,21):
    # define model
    pretrained_model = EfficientNet.from_pretrained('efficientnet-b0')
    pretrained_model._fc = nn.Linear(1280, 2)
    model = pretrained_model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters())

    # training
    torch.manual_seed(num*100) #seeds be 100, 200, 300, 400, 500...
    # model.train()
    print(f'start train {num}th model. seed={num*100}')
    for epoch in range(10): # epoch 10으로 설정
        running_loss = 0.0
        for i, data in enumerate(train_dataloader):
            images, labels = data[0].to(device), data[1].to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            if i % 100 == 99:
                print(f'[epoch: {epoch+1} / batch: {i+1:3d}] loss: {running_loss/100:.4f}')
                running_loss = 0.0
    print(f'Finished Training_{num}')
    # model save
    path = f'./SVD/model_mel-spec_{num}.pth'
    torch.save(model.state_dict(), path)
    print(f'Model has saved as ./SVD/model_mel-spec_{num}.pth')
    print('---------------------------------------------------------------------------')

In [ ]:
# A function that load saved CNN models (effnet) and creates a list of prediction labels (cnn_preds).
def CNN_pred(model, loader):
    y_true = []
    y_pred = []
    model.eval()
    with torch.no_grad():
        for data in loader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            y_true.extend(labels.tolist())
            y_pred.extend(preds.tolist())
    return [y_true, y_pred]

# function get cm and return acc, f1
def metrics(cm):
    tn, fp, fn, tp = cm.ravel()
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    recall = (tp) / (tp + fn)
    precision = (tp) / (tp + fp)
    f1 = (2 * recall * precision) / (recall + precision)
    return accuracy, f1

In [ ]:
# Recalling previously stored models 1 to 20,
# Since you have to average 1 to 20, put it in the form of a dictionary in cnn_preds.
cnn_preds = {}
for num in range(1,21):
    pretrained_model = EfficientNet.from_name('efficientnet-b0')
    pretrained_model._fc = nn.Linear(1280, 2)
    state_dict = torch.load(f'./SVD/model_mel-spec_{num}.pth')
    pretrained_model.load_state_dict(state_dict)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = pretrained_model.to(device)

    y_true, cnn_pred = CNN_pred(model, test_dataloader)
    # Store the predicted value in a dictionary with keys such as 'cnn_pred_1', 'cnn_pred_2', etc
    cnn_preds[f'cnn_pred_{num}'] = cnn_pred


print(y_true) # testset
print(cnn_preds) # dictionary that has 20 predictions.

#### Applicated HardVoting implementation

In [ ]:
for n in range(1,21):
    y_preds = np.vstack(list(cnn_preds4.values())[:n]) # Since it may not be good to mix a lot of models unconditionally, I checked from 1 to n for confirmation.
    y_pred_avg = np.mean(y_preds, axis=0)
    # Determining the final prediction class based on the average probability (binary classification based on 0.3, 0.4, 0.5, etc.)
    # Combine dictionaries to create a single prediction label
    # Determining the final prediction based on the average of the predictions (means greater than 0.45 => 1)
    y_pred_final = (y_pred_avg > 0.45).astype(int)
    cm = confusion_matrix(y_true, y_pred_final)
    print(f'Voting Case : by combine 1~{n}models :')
    print(cm)
    accuracy, f1 = metrics(cm)
    print(accuracy, f1)
    print('--------------------------------------------------')